新增一筆資料

In [4]:
# 添加一筆資料
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  # Neo4j Bolt URL
username = "neo4j"
password = "sam112233"

# 連接到數據庫
driver = GraphDatabase.driver(uri, auth=(username, password))

def add_person(tx, name):
    query = (
        "CREATE (p:Person {name: $name}) "
        "RETURN p"
    )
    return tx.run(query, name=name).single()[0]

# 新增節點
with driver.session() as session:
    person = session.write_transaction(add_person, "Alice3")
    print(f"Created: {person['name']}")

# 關閉數據庫連接
driver.close()


Created: Alice3


/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_499/284930031.py:20: DeprecationWarning: write_transaction has been renamed to execute_write
  person = session.write_transaction(add_person, "Alice3")


`查詢資料`

In [5]:
# 查詢資料
def find_person(tx, name):
    query = (
        "MATCH (p:Person {name: $name}) "
        "RETURN p.name AS name"
    )
    result = tx.run(query, name=name).single()
    return result["name"] if result else "No such person found"

# 查詢節點
with driver.session() as session:
    person_name = session.read_transaction(find_person, "Alice2")
    print(f"Found: {person_name}")


Found: Alice2


/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_499/1429163631.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  person_name = session.read_transaction(find_person, "Alice2")


建立多筆具有關係的資料

In [6]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  # Neo4j Bolt URL
username = "neo4j"
password = "sam112233"

# 連接到資料庫
driver = GraphDatabase.driver(uri, auth=(username, password))


def add_people_and_relationships(tx):
    # 建立多個人物節點和它們之間的關係
    query = """
        CREATE (a:Person {name: 'Alice'})
        CREATE (b:Person {name: 'Bob'})
        CREATE (c:Person {name: 'Carol'})
        CREATE (d:Person {name: 'Dave'})
        CREATE (e:Person {name: 'Eve'})
        
        CREATE (a)-[:FRIENDS_WITH]->(b)
        CREATE (a)-[:COLLEAGUES_WITH]->(c)
        CREATE (b)-[:FRIENDS_WITH]->(d)
        CREATE (c)-[:COLLEAGUES_WITH]->(d)
        CREATE (d)-[:FRIENDS_WITH]->(e)
        CREATE (e)-[:COLLEAGUES_WITH]->(a)
        
        RETURN a, b, c, d, e
    """
    results = tx.run(query)
    return results.single()  # Return the created nodes


# 運行事務並列印結果
with driver.session() as session:
    people = session.write_transaction(add_people_and_relationships)
    print("Created the following nodes:")
    for person in people:
        print(f"Created: {person['name']}")

# 關閉資料庫連接
driver.close()

/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_499/1679174612.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  people = session.write_transaction(add_people_and_relationships)
Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv6Address(('::1', 7687, 0, 0)))


Created the following nodes:
Created: Alice
Created: Bob
Created: Carol
Created: Dave
Created: Eve


透過添加關係和節點束性來建立多個人物節點間的多種關係

In [7]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "sam112233"

# 連接到資料庫
driver = GraphDatabase.driver(uri, auth=(username, password))

def add_people_and_complex_relationships(tx):
    # 透過添加關係和節點束性來建立多個人物節點間的多種關係
    query = """
    CREATE (a:Person {name: 'Alice', age: 30, occupation: 'Engineer'})
    CREATE (b:Person {name: 'Bob', age: 25, occupation: 'Designer'})
    CREATE (c:Person {name: 'Carol', age: 33, occupation: 'Manager'})
    CREATE (d:Person {name: 'Dave', age: 45, occupation: 'Developer'})
    CREATE (e:Person {name: 'Eve', age: 29, occupation: 'Analyst'})
    
    CREATE (a)-[:FRIENDS_WITH {since: 2020}]->(b)
    CREATE (a)-[:COLLEAGUES_WITH {since: 2018}]->(c)
    CREATE (b)-[:FRIENDS_WITH {since: 2021}]->(d)
    CREATE (c)-[:COLLEAGUES_WITH {since: 2019}]->(d)
    CREATE (d)-[:FRIENDS_WITH {since: 2019}]->(e)
    CREATE (e)-[:COLLEAGUES_WITH {since: 2021}]->(a)
    CREATE (a)-[:MENTOR_OF {since: 2022}]->(e)
    CREATE (b)-[:WORKS_WITH {on: 'Project X'}]->(c)
    CREATE (d)-[:LIVES_NEAR {distance: '5km'}]->(b)
    
    RETURN a, b, c, d, e
    """
    results = tx.run(query)
    # 傳回創建的節點和關係
    return results.single()

# 運行並輸出結果
with driver.session() as session:
    entities = session.write_transaction(add_people_and_complex_relationships)
    print("Created the following nodes and relationships:")
    for entity in entities:
        print(f"Created: {entity['name']} with attributes {dict(entity)}")

# 關閉資料庫連接
driver.close()


Created the following nodes and relationships:
Created: Alice with attributes {'occupation': 'Engineer', 'name': 'Alice', 'age': 30}
Created: Bob with attributes {'occupation': 'Designer', 'name': 'Bob', 'age': 25}
Created: Carol with attributes {'occupation': 'Manager', 'name': 'Carol', 'age': 33}
Created: Dave with attributes {'occupation': 'Developer', 'name': 'Dave', 'age': 45}
Created: Eve with attributes {'occupation': 'Analyst', 'name': 'Eve', 'age': 29}


/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_499/1328549409.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  entities = session.write_transaction(add_people_and_complex_relationships)
